# Chapter 5 Exercises

1. _What is the fundamental idea behind Support Vector Machines?_
 
 wide margin separation, for better generalization


2. _What is a support vector?_

 data point that defines the outer margin


3. _Why is it important to scale the inputs when using SVMs?_

 they are sensitive to scale, and need bias removed


4. _Can an SVM classifier output a confidence score when it classifies an instance? What about a probability?_ 

 yes. no
 
 
5. _Should you use the primal or the dual form of the SVM problem to train a model on a training set with millions of instances and hundreds of features?_ 

 (skipped, optional section, question omitted in 3rd edition)
 
 
6. _Say you’ve trained an SVM classifier with an RBF kernel, but it seems to underfit the training set. Should you increase or decrease γ (gamma)? What about C?_ 

 increase. increase


7. _How should you set the QP parameters (H, f, A, and b) to solve the soft margin linear SVM classifier problem using an off-the-shelf QP solver?_ 

 (skipped, optional section, question omitted in 3rd edition)

## 8.
_Train a LinearSVC on a linearly separable dataset. Then train an SVC and a SGDClassifier on the same dataset. See if you can get them to produce roughly the same model._

In [53]:
import numpy as np
from sklearn import datasets
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

iris = datasets.load_iris()
list(iris.keys())
X = iris["data"][:, 3:]  # petal width
y = (iris["target"] == 2).astype(int)  # 1 if Iris setosa, else 0


linsvc_class = Pipeline([
        ("scaler", StandardScaler()),
        ("linear_svc", LinearSVC(C=1, loss="squared_hinge", random_state=42)),
    ])

svc_class = Pipeline([
        ("scaler", StandardScaler()),
        ("svc", SVC(random_state=42, kernel='linear', C=1)),
    ])

sgd_class = Pipeline([
        ("scaler", StandardScaler()),
        ("sgd", SGDClassifier(random_state=42, loss="squared_hinge", alpha=1/(X.shape[0] * 1), max_iter=1000000))#, early_stopping=True)),
    ])

In [8]:
from sklearn.model_selection import cross_val_score

def display_scores(estimator, scores):
    print("\n")
    print(type(estimator).__name__)
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())
        
def print_cv_scores(estimators, X, y):
    scores_and_estimators = []
    for e in estimators:
        scores = cross_val_score(e, X, y, cv=3, verbose=3)
        scores_and_estimators.append((scores.mean(), scores, e))
        
    scores_and_estimators.sort(key = lambda x: x[0], reverse=True)
    for mean, scores, e in scores_and_estimators:
        display_scores(e, scores)

In [ ]:
estimators = [svc_class, linsvc_class, sgd_class]
print_cv_scores(estimators, X, y)

In [55]:
from sklearn.model_selection import cross_val_predict

y_cvp_linsvc = cross_val_predict(linsvc_class, X, y, cv=5)
#print(y_cvp_linsvc)

y_cvp_svc = cross_val_predict(svc_class, X, y, cv=5)
#print(y_cvp_svc)

y_cvp_sgd = cross_val_predict(sgd_class, X, y, cv=5)
#print(y_cvp_sgd)

print ((y_cvp_linsvc - y_cvp_svc).sum())
print ((y_cvp_linsvc - y_cvp_sgd).sum())

1
0


/Users/chrismarch/OtherRepos/ml/ml_env/lib/python3.8/site-packages/sklearn/linear_model/_stochastic_gradient.py:696: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


## 9.
_Train an SVM classifier on the MNIST dataset. Since SVM classifiers are binary classifiers, you will need to use one-versus-the-rest to classify all 10 digits. You may want to tune the hyperparameters using small validation sets to speed up the process. What accuracy can you reach?_

In [1]:
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784', version=1, as_frame=False)

In [2]:
import joblib
joblib.dump(mnist, "minst.pkl")

['minst.pkl']

In [2]:
import joblib
mnist = joblib.load("minst.pkl")

In [3]:
import numpy as np

X, y = mnist["data"], mnist["target"]
y = y.astype(np.uint8)
X_train, X_test, y_train, y_test = X[:60000], X[60000:], y[:60000], y[60000:]

In [4]:
import numpy as np
from sklearn import datasets
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

linsvc_clf = LinearSVC(C=.09, max_iter=2000, dual=False, random_state=42) # ,  hinge unsuppored with dual=False loss="hinge", penalty='l2',

# too slow for MINST (M=70,000)
svc_clf = Pipeline([
        #("scaler", StandardScaler()),
        ("svc", SVC(random_state=42, kernel='linear', C=1)),
    ])

sgd_clf = SGDClassifier(random_state=42)
sgd_tune_clf = SGDClassifier(random_state=42, alpha=.0001, max_iter=900)#, early_stopping=True, n_iter_no_change=100, alpha=.0002, max_iter=900)#, loss="squared_hinge", alpha=1/(X.shape[0] * 1), max_iter=1000000))#, early_stopping=True)),

scaler = StandardScaler()

In [5]:
X_train_scaled = scaler.fit_transform(X_train.astype(np.float64))

In [32]:
X_train_sm, y_train_sm = X_train_scaled[:10000], y_train[:10000]

In [13]:
#print_cv_scores([linsvc_clf], X_train_scaled, y_train)
print_cv_scores([sgd_tune_clf], X_train_scaled, y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/Users/chrismarch/OtherRepos/ml/ml_env/lib/python3.8/site-packages/sklearn/linear_model/_stochastic_gradient.py:696: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  3.7min remaining:    0.0s


[CV] END ................................ score: (test=0.899) total time= 3.7min


/Users/chrismarch/OtherRepos/ml/ml_env/lib/python3.8/site-packages/sklearn/linear_model/_stochastic_gradient.py:696: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  7.4min remaining:    0.0s


[CV] END ................................ score: (test=0.891) total time= 3.7min
[CV] END ................................ score: (test=0.902) total time= 3.9min


SGDClassifier
Scores: [0.8987 0.8912 0.9022]
Mean: 0.8973666666666666
Standard deviation: 0.004588633299312078


/Users/chrismarch/OtherRepos/ml/ml_env/lib/python3.8/site-packages/sklearn/linear_model/_stochastic_gradient.py:696: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed: 11.3min finished


### CV Scores

LinearSVC C=.1  
Scores: [0.90555 0.90555 0.9092 ]
Mean: 0.9067666666666666
Standard deviation: 0.0017206265008872855

SGDClassifier  
Scores: [0.8983 0.891  0.9018]
Mean: 0.8970333333333333
Standard deviation: 0.004499135719471274

SGDClassifier tune  
Scores: [0.8987 0.8912 0.9022]
Mean: 0.8973666666666666
Standard deviation: 0.004588633299312078

#### 10k subset

SGD  
Scores: [0.89172166 0.90939094 0.8889889 ]
Mean: 0.8967004978842216
Standard deviation: 0.009042583092445447

LSVC c=.1  
Scores: [0.87822436 0.89288929 0.87668767]
Mean: 0.8826004376082479
Standard deviation: 0.007302314581355561

In [53]:
from sklearn.model_selection import GridSearchCV

linsvc_param_grid = [
    {'C' : [.0925, .09, .088], 'max_iter' : [2000,2200,1850]}#,'loss' : ['hinge', 'squared_hinge']},
  ]

sgd_param_grid = [
    {'alpha' : [0.0001, 0.00005, 0.0015]}#, 'max_iter' : [900,920,850]}#,'loss' : ['hinge', 'squared_hinge']},
  ]

grid_search = GridSearchCV(sgd_clf, sgd_param_grid, cv=3,
                           #return_train_score=True, 
                           verbose=2)

In [54]:
pipe_out = grid_search.fit(X_train_scaled, y_train)
print(grid_search.best_params_)

Fitting 3 folds for each of 3 candidates, totalling 9 fits


/Users/chrismarch/OtherRepos/ml/ml_env/lib/python3.8/site-packages/sklearn/linear_model/_stochastic_gradient.py:696: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


[CV] END .......................................alpha=0.0001; total time= 3.9min


/Users/chrismarch/OtherRepos/ml/ml_env/lib/python3.8/site-packages/sklearn/linear_model/_stochastic_gradient.py:696: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


[CV] END .......................................alpha=0.0001; total time= 4.0min


...

[CV] END ........................................alpha=5e-05; total time= 4.4min


/Users/chrismarch/OtherRepos/ml/ml_env/lib/python3.8/site-packages/sklearn/linear_model/_stochastic_gradient.py:696: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


[CV] END ........................................alpha=5e-05; total time= 4.6min


/Users/chrismarch/OtherRepos/ml/ml_env/lib/python3.8/site-packages/sklearn/linear_model/_stochastic_gradient.py:696: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


[CV] END ........................................alpha=5e-05; total time=20.9min
[CV] END .......................................alpha=0.0015; total time=  54.5s
[CV] END .......................................alpha=0.0015; total time=  52.0s
[CV] END .......................................alpha=0.0015; total time=  54.9s
{'alpha': 0.0015}


In [6]:
sgd_clf.fit(X_train_scaled, y_train)

SGDClassifier(random_state=42)

In [11]:
y_predict = sgd_clf.predict(X_test)

In [12]:
from sklearn.metrics import f1_score
f1_score(y_test, y_predict, average="macro")

0.8161582089093642

## test scores
LinearSVC(C=0.09, dual=False, max_iter=2000, random_state=42)
0.768437796254467

SGDClassifier(alpha=0.0002, early_stopping=True, max_iter=900,
              n_iter_no_change=100, random_state=42)
0.7758249617681228

default sgd
0.8161582089093642

## 10.
_Train and fine-tune an SVM regressor on the California housing dataset. You can use the original dataset rather than the tweaked version we used in Chapter 2. The original dataset can be fetched using sklearn.datasets.fetch_california_housing(). The targets represent hundreds of thousands of dollars. Since there are over 20,000 instances, SVMs can be slow, so for hyperparameter tuning you should use far fewer instances (e.g., 2,000), to test many more hyperparameter combinations. What is your best model’s RMSE?_ (3rd edition elaboration) 

In [5]:
import sklearn.datasets
dataset = sklearn.datasets.fetch_california_housing()

In [7]:
import pandas as pd
df = pd.DataFrame(data=dataset.data, columns=dataset.feature_names)
df.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25


In [8]:
dataset

{'data': array([[   8.3252    ,   41.        ,    6.98412698, ...,    2.55555556,
           37.88      , -122.23      ],
        [   8.3014    ,   21.        ,    6.23813708, ...,    2.10984183,
           37.86      , -122.22      ],
        [   7.2574    ,   52.        ,    8.28813559, ...,    2.80225989,
           37.85      , -122.24      ],
        ...,
        [   1.7       ,   17.        ,    5.20554273, ...,    2.3256351 ,
           39.43      , -121.22      ],
        [   1.8672    ,   18.        ,    5.32951289, ...,    2.12320917,
           39.43      , -121.32      ],
        [   2.3886    ,   16.        ,    5.25471698, ...,    2.61698113,
           39.37      , -121.24      ]]),
 'target': array([4.526, 3.585, 3.521, ..., 0.923, 0.847, 0.894]),
 'frame': None,
 'target_names': ['MedHouseVal'],
 'feature_names': ['MedInc',
  'HouseAge',
  'AveRooms',
  'AveBedrms',
  'Population',
  'AveOccup',
  'Latitude',
  'Longitude'],
 'DESCR': '.. _california_housing_dataset:\n